## Prepare data reader and model execution context

In [8]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.etl.blockbatch.construct import construct_data_readers
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "refined_traces"

# Define the input data range.
readers: list[DataReader] = construct_data_readers(
    blockbatch_request=BlockBatchRequest.build(
        chains=["op"],
        range_spec="@20241118:+1",
    ),
    models=[model_name],
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, aux_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-01-08 15:05:29 [info     ] prepared 91 input batches.     filename=byblock.py lineno=83 process=19105
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain', value='op'),
                                      PartitionColumn(name='dt',
                                                      value='2024-11-17')]),
           read_from=DataLocation.GCS,
           dataset_paths={'ingestion/blocks_v1': ['gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-17/000128100000.parquet'],
                          'ingestion/traces_v1': ['gs://oplabs-tools-data-sink/ingestion/traces_v1/chain=op/dt=2024-11-17/000128100000.parquet'],
                          'ingestion/transactions_v1': ['gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-17/000128100000.parquet']},
           inputs_ready=True,
           extra_marker_data={'max_block': 128102000,
                              'min_block': 128100000,
                              'num_blocks'

## Execute the model

It's up to you how the model manipulates the data. 

Develop the various steps in notebook cells and then copy the final code over to the model function.

In [2]:
refined_txs_table = aux_views["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_traces_projection = aux_views["refined_traces/traces_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
    },
)

traces_with_gas_used = aux_views["refined_traces/traces_with_gas_used"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_traces_projection": refined_traces_projection,
    },
)

traces_txs_join = aux_views["refined_traces/traces_txs_join"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "traces_with_gas_used": traces_with_gas_used,
        "refined_transactions_fees": refined_txs_table,
    },
)

2025-01-08 15:02:36 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=19105
2025-01-08 15:02:36 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=19105
2025-01-08 15:02:36 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=19105 template=refined_transactions_fees
2025-01-08 15:02:40 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=19105
2025-01-08 15:02:40 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=19105
2025-01-08 15:02:40 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=19105 template=refined_traces/traces_projection
2025-01-08 15:02:43 [info     ] duck db size: 14.2MB           filename=client.py lineno=36 process=19105
2025-01-08 15:02:43 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=19105 templa

In [5]:
ctx.client.sql("""
SELECT * FROM refined_traces__traces_txs_join
WHERE transaction_hash = '0x348e71e1290557d15582057f46562e1b05b6ab821d9e80015b2cd186b622ed47'
ORDER BY trace_address
LIMIT 10
""").show(max_rows=100)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬────────────┬───────────┬────────────────────┬─────────────────┬───────────────┬─────────────┬──────────────────────┬───────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────────────────

In [9]:
ctx.client.sql(f"""DESCRIBE {refined_txs_table}""").show(max_rows=100)

┌─────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │  column_type   │  null   │   key   │ default │  extra  │
│               varchar               │    varchar     │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                  │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                               │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                            │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                             │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ nonce                               │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index                   │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ from_address                